In [1]:

#Importing libraries
import requests
import os
import json
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup  
import time
import numpy as np
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import traceback
from webdriver_manager.chrome import ChromeDriverManager  


## Creating a Chrome manager instance to get the links of articles from Journal of corporate finance 

In [3]:

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
driver.set_page_load_timeout(20)



Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_win32.zip
Unpack archive C:\Users\venub\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.zip


C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [4]:
links=['https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/issues?page=2','https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/issues?page=2']


In [5]:

def get_url_and_wait_for_page_load(_driver, url):
    _driver.get(url)
    WebDriverWait(driver, 30).until(
        EC.visibility_of_all_elements_located((By.XPATH, "//div[@class = 'usabilla_live_button_container']")))


## Getting the issue links in the volumes 

In [7]:
volume_links, issue_parse_failed = [], []  # you can reprocess failed list if you wish
for link in links:
    try:
        print "Looking for volume links @ {}".format(link)
        get_url_and_wait_for_page_load(driver, link)
        driver.execute_script('var accordions = document.getElementsByClassName("accordion-panel-title"); '
                              'for(accordion in accordions) if(accordion > 1) accordions[accordion].click();')
        volume_links += [link.get_attribute("href")
                         for link in driver.find_elements_by_xpath("//a[contains(@class, 'js-issue-item-link')]")]
        print "Total volume links: {}".format(len(volume_links))
    except:
        print "Failed to get volume links @ {}".format(link)
        issue_parse_failed.append(link)

Looking for volume links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/issues?page=2
Total volume links: 4


## Getting all the article links

In [8]:
article_links, volume_parse_failed = [], []  # you can reprocess failed list if you wish
for volume_link in volume_links:
    try:
        print ("Looking for article links @ {}".format(volume_link))
        get_url_and_wait_for_page_load(driver, volume_link)
        article_links += [link.get_attribute("href")
                          for link in driver.find_elements_by_xpath("//a[contains(@class, 'article-content-title')]")]
        print ("Total article links: {}".format(len(article_links)))
    except:
        
        print ("Failed to get article links @ {}".format(volume_link))
        volume_parse_failed.append(volume_link)

Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/64/suppl/C
Total article links: 28
Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/62/suppl/C
Total article links: 80
Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/61/suppl/C
Total article links: 93
Looking for article links @ https://www-sciencedirect-com.dcu.idm.oclc.org/journal/journal-of-corporate-finance/vol/60/suppl/C
Total article links: 134


## Opening a new chrome instance to scrape the data from the articles

In [9]:
options = webdriver.ChromeOptions() 



In [11]:
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [12]:
driver = webdriver.Chrome(options=options, executable_path=r"C:/selenium/chromedriver.exe")


## Initializing the home window 

In [44]:
driver.switch_to_window(driver.window_handles[0])

C:\Users\venub\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


## Defining an empty list to store the data

In [13]:
corporate_finance_list=[]

In [ ]:
for link in article_links:
    driver.get(link)
    window0  = driver.current_window_handle
    sa={}
    sleep(7)
    try:
        sa['title']=driver.find_element_by_xpath('//*[@id="screen-reader-main-title"]/span').text
        sa["link"]=driver.find_element_by_xpath('//*[@id="doi-link"]/a[1]').text
        sa["publishment_details"]=driver.find_element_by_xpath('//*[@id="publication"]/div[2]/div').text
        sa["abstract"]=driver.find_element_by_xpath('/html/body/div[4]/div/div/section/div/div/div/div[2]/article/div[5]/div').text
        sa["highlights"]=driver.find_element_by_xpath('//*[@id="ab0010"]').text

        
        sa["citations"]=driver.find_element_by_xpath('//*[@id="citing-articles-header"]').text
    except:
        print("elements missing")
    driver.execute_script("window.scrollTo(0,400);")
    try:
        sa["figures"]=driver.find_element_by_xpath('//*[@id="toc-figures"]/h2').text
        sa["tables"]=driver.find_element_by_xpath('//*[@id="toc-tables"]/h2').text
    except:
        print("elements missing")
    try:
        sa["references"]=driver.find_element_by_xpath('/html/body/div/div/div/section/div/div/div/div[2]/article/section[3]').text
    except:
        print("references not found")
    driver.execute_script("window.scrollTo(400,700)")
    try:
        sa["body"]= WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="body"]'))).text
    except:
        print("body not found")
    try:
        sa["k/c"]=driver.find_element_by_xpath('/html/body/div[4]/div/div/section/div/div/div/div[2]/article/div[6]/div[1]').text
        sa["c/k"]=driver.find_element_by_xpath('/html/body/div[4]/div/div/section/div/div/div/div[2]/article/div[6]/div[2]').text
    except:
        print("classifications missing")
        
    driver.execute_script("window.scrollTo(700,0);")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[1]/span').click()
        sa["author1"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author1-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author1-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author1-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author1")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[2]/span').click()
        sa["author2"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author2-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author2-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author2-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author2")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[3]/span').click()
        sa["author3"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author3-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author3-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author3-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author3")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[4]/span').click()
        sa["author4"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author4-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author4-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author4-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author4")
    try:
        driver.find_element_by_xpath('//*[@id="author-group"]/a[5]/span').click()
        sa["author5"]=driver.find_element_by_xpath('//*[@id="workspace-author"]/div[1]').text
        sleep(2)
        iframe = driver.find_element_by_xpath('//iframe[contains(@title, "Author Profile")]')
        driver.switch_to.frame(iframe)
        sa["author5-h-index"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[1]/data').text
        sa["author5-citations"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[3]/ul/li[2]/data').text
        sa["author5-university"]=driver.find_element_by_xpath('//*[@id="profile-preview-page"]/div/div/div/div[1]/div[2]/span').text
        driver.switch_to_window(window0)
    except:
        print("no author5")
    corporate_finance_list.append(sa)
    


## saving the list into a dataframe

In [16]:
corporate_finance=pd.DataFrame(corporate_finance_list)

## Feature engineering the data 

In [17]:
corporate_finance['tables']=corporate_finance['tables'].str.extract('(\d+)')
corporate_finance['figures']=corporate_finance['figures'].str.extract('(\d+)')
a=corporate_finance['publishment_details'].str.split(',',expand=True)
corporate_finance['volume']=a[0]
corporate_finance['published time']=a[1]
corporate_finance['article number']=a[2]
intro=corporate_finance['body'].str.split('\n2. ',expand=True)
corporate_finance["introduction"]=intro[0]
corporate_finance["Citations"]=corporate_finance['citations'].str.extract('(\d+)')

In [108]:
body=corporate_finance['body']

In [81]:
C1=corporate_finance["body"].str.split('Conclusion\n',expand=True)

In [82]:
C1 = C1.replace(np.nan, '', regex=True)

In [84]:
C2=corporate_finance["body"].str.split('Conclusions\n',expand=True)

In [85]:
C2 = C2.replace(np.nan, '', regex=True)

In [87]:
corporate_finance["Conclusion"]=C1[1].str.cat(C2[1],sep="")

In [89]:
corporate_finance

,title,link,publishment_details,abstract,figures,tables,references,body,k/c,c/k,...,author4-university,author5,author5-h-index,author5-citations,author5-university,volume,published time,article number,introduction,Conclusion
0,Editorial Board,https://doi.org/10.1016/S0929-1199(08)00103-X,"Volume 14, Issue 5, December 2008, Page IFC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,NaN,
1,You can't take it with you: Sunset provisions ...,https://doi.org/10.1016/j.jcorpfin.2008.08.006,"Volume 14, Issue 5, December 2008, Pages 499-511",Abstract\nCompany stock option plans have dive...,3,7,"References\nAmmann and Seiz, 2004\nAmmann M., ...",1. Introduction\nThis study investigates how c...,JEL classification\nG32J33,Keywords\nExecutive stock optionsVestingForfei...,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nThis study investigates how c...,This paper investigates sunset provisions of e...
2,Board compensation practices and agency costs ...,https://doi.org/10.1016/j.jcorpfin.2008.09.004,"Volume 14, Issue 5, December 2008, Pages 512-531",Abstract\nExtant theory and empirical evidence...,NaN,NaN,"References\nAltman, 1968\nAltman E.\nFinancial...",1. Introduction\nThe literature on agency prob...,JEL classification\nG34G32J33,Keywords\nDirector incentivesCost of debtCorpo...,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nThe literature on agency prob...,In this paper we examine how equity-based comp...
3,The determinants of board composition in a tra...,https://doi.org/10.1016/j.jcorpfin.2008.09.005,"Volume 14, Issue 5, December 2008, Pages 532-549",NaN,2,9,"References\nAbe, 2003\nAbe N.\nKeieisha turnov...",1. Introduction\nDespite the fact that the gen...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nDespite the fact that the gen...,"In this paper, we conducted a comprehensive an..."
4,Protective governance choices and the value of...,https://doi.org/10.1016/j.jcorpfin.2008.09.011,"Volume 14, Issue 5, December 2008, Pages 550-566",NaN,NaN,NaN,"References\nAdams and Ferreira, 2007\nAdams R....",1. Introduction\nMany publicly traded corporat...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Volume 14,Issue 5,December 2008,1. Introduction\nMany publicly traded corporat...,Our study of the value and likelihood of acqui...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,Board effectiveness and board dissent: A model...,https://doi.org/10.1016/S0929-1199(97)00009-6,"Volume 4, Issue 1, March 1998, Pages 53-70","Abstract\nTo date, there has been little model...",NaN,NaN,"References\nBerle and Means, 1932\nBerle, A.A....",1. Introduction\nThe relation between sharehol...,Keywords\nBoards of directorsManagerial discip...,JEL classification\nG32,...,NaN,NaN,NaN,NaN,NaN,Volume 4,Issue 1,March 1998,1. Introduction\nThe relation between sharehol...,"To date, there has been little modeling of the..."
381,An options-based model of equilibrium credit r...,https://doi.org/10.1016/S0929-1199(97)00010-2,"Volume 4, Issue 1, March 1998, Pages 71-85",Abstract\nThis paper applies options theory to...,5,1,"References\nBerger and Udell, 1992\nA.N Berger...",1. Introduction\nPrices in well-functioning ma...,Keywords\nEquilibrium credit rationingOption p...,JEL classification\nD82G13G30,...,NaN,NaN,NaN,NaN,NaN,Volume 4,Issue 1,March 1998,1. Introduction\nPrices in well-functioning ma...,Interpreting the Stiglitz and Weiss (1981)mode...
382,Internal financing of multinational subsidiari...,https://doi.org/10.1016/S0929-1199(97)00011-4,"Volume 4, Issue 1, March 1998, Pages 87-106",Abstract\nMultinational subsidiaries are gener...,1,NaN,"References\nChowdhry and Nanda, 1994\nB. Chowd...",1. Introduction\nSubsidiaries of multinational...,Keywords\nMultinational capital structureInter...,JEL classification\nG3G32F3F30,...,NaN,NaN,NaN,NaN,NaN,Volume 4,Issue 1,March 1998,1. Introduction\nSubsidiaries of multination

## Saving the data into a csv

In [ ]:
corporate_finance.to_csv("Corporate_finance.csv")